In [36]:
import matplotlib
import pathlib
import numpy as np

import mne
print(mne.__version__)

0.23.0


In [37]:
#Creating event id Dictionary
event_id = {
    "taskstart" : '9',
    "cue" : "1",
    "go" : "2",
    "button press" : "5",
    "no-go" : "4",
    "task end": "10",
    "error 1" : "3",
    "error 2" : "6",
    "error 3" : "7",
    "error 4" : "8",
    "error 5" : "11"
}
event_id

{'taskstart': '9',
 'cue': '1',
 'go': '2',
 'button press': '5',
 'no-go': '4',
 'task end': '10',
 'error 1': '3',
 'error 2': '6',
 'error 3': '7',
 'error 4': '8',
 'error 5': '11'}

In [38]:
#to fix event ids
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]
def fix_event_ids(epochs, events):
    for i in range(epochs.events.shape[0]):
        epochs.events[i][2] = int(get_keys_from_value(events[1], epochs.events[i][2])[0])

In [39]:
#get type of channels from tsv file
import pandas as pd

#path Sahil
path_channels_tsv = '../../data\sub-AB10_eeg_sub-AB10_task-gonogo_run-1_channels.tsv'
#path Arush
#path_channels_tsv = 'data/other subjects set files/sub-AB10_eeg_sub-AB10_task-gonogo_run-1_channels.tsv'

channel_types_tsv = pd.read_csv(path_channels_tsv, delimiter = '\t')

def get_channel_types_dic(channel_types_tsv):
    channel_types = {}
    for i in range(channel_types_tsv.shape[0]):
        channel_types[channel_types_tsv['name'][i]] = channel_types_tsv['type'][i].lower()
    reassign_dic = {'HEO':'eog', 'VEO':'eog', 'R-Dia-X-(mm)':'misc', 'R-Dia-Y-(mm)':'misc'}
    for k in list(reassign_dic.keys()):
        if k in list(channel_types):
            channel_types[k] = reassign_dic[k]
    return channel_types

channel_types = get_channel_types_dic(channel_types_tsv)

In [40]:
#To find and mark bad components
def mark_artifacts(ica, raw):
    ica.exclude = []
    ica.detect_artifacts(raw);
    eeg_bads = list(ica.exclude)
    ecg_bads = ica.find_bads_ecg(raw)[0];
    eog_bads = ica.find_bads_eog(raw)[0];
    ica.exclude = list(set(eeg_bads+ecg_bads+eog_bads))

In [41]:
def clean_raw(unclean_raw):
    
    unclean_raw.set_channel_types(channel_types)
    unclean_raw.filter(0.5, 60);
    return unclean_raw.copy()

In [42]:
def apply_ica(raw):
    
    ica = mne.preprocessing.ICA(n_components = 20, random_state = 42)
    ica.fit(raw);
    mark_artifacts(ica, raw)
    ica.apply(raw, exclude = ica.exclude)

In [43]:
def get_data(f):
    
    #path Sahil
    path = '../../EEG original dataset/sub-AB{}/eeg/sub-AB{}_task-gonogo_run-{}_eeg.set'.format(f[0:-2], f[0:-2], f[-1])
    #path Arush
    'data/other subjects set files/sub-AB{}_eeg_sub-AB{}_task-gonogo_run-{}_eeg.set'.format(f[0:-2], f[0:-2], f[-1])
    
    unclean_raw = mne.io.read_raw_eeglab(path, eog=(), preload=True, uint16_codec=None, verbose=None)
    
    raw = clean_raw(unclean_raw)
    apply_ica(raw)
    
    events = mne.events_from_annotations(raw) #generating events
    epochs = mne.Epochs(raw,
                   events = events[0],
                   event_id = events[1],)
    fix_event_ids(epochs, events)
    
    return epochs.get_data()

In [57]:
#The files we will be loading
#files = ['10_1', '10_2', '12_1', '12_2', '13_1', '13_2', '14_1', '14_2', '15_1', '15_2', '16_1', '16_2', '21_1', '21_2', '22_1', '22_2', '23_1', '23_2', '28_1', '28_2', '31_1', '31_2', '32_1', '32_2', '33_1', '33_2', '34_1', '34_2', '35_1', '35_2', '46_1', '46_2', '47_1', '47_2', '48_1', '48_2', '49_1', '49_2', '50_1', '50_2', '69_1', '69_2', '70_1', '70_2', '71_1', '71_2', '72_1', '72_2']
#files = ['10_1', '10_2', '12_1', '12_2', '13_1', '13_2', '14_1', '14_2', '15_1', '15_2', '28_1', '28_2', '31_1', '31_2', '32_1', '32_2']
#files = ['10_1', '10_2']
files = ['4_1', '4_2', '7_1', '7_2', '8_1', '8_2', '9_1', '9_2', '10_1', '10_2', '11_1', '11_2', '12_1', '12_2', '13_1', '13_2', '14_1', '14_2', '15_1', '15_2', '16_1', '16_2', '17_1', '17_2', '19_1', '19_2', '20_1', '20_2', '21_1', '21_2', '22_1', '22_2', '23_1', '23_2', '25_1', '25_2', '26_1', '26_2', '28_1', '28_2', '31_1', '31_2', '32_1', '32_2', '33_1', '33_2', '34_1', '34_2', '35_1', '35_2', '36_1', '36_2', '37_1', '37_2', '41_1', '41_2', '42_1', '42_2', '43_1', '43_2', '44_1', '44_2', '45_1', '45_2', '46_1', '46_2', '47_1', '47_2', '48_1', '48_2', '49_1', '49_2', '50_1', '50_2', '51_1', '51_2', '52_1', '52_2', '53_1', '53_2', '54_1', '54_2', '55_1', '55_2', '56_1', '56_2', '57_1', '57_2', '58_1', '58_2', '59_1', '59_2', '60_1', '60_2', '61_1', '61_2', '62_1', '62_2', '63_1', '63_2', '64_1', '64_2', '65_1', '65_2', '66_1', '66_2', '67_1', '67_2', '68_1', '68_2', '69_1', '69_2', '70_1', '70_2', '71_1', '71_2', '72_1', '72_2', '73_1', '73_2', '74_1', '74_2', '75_1', '75_2', '76_1', '76_2', '77_1', '77_2', '78_1', '78_2', '79_1', '79_2', '80_1', '80_2', '81_1', '81_2', '82_1', '82_2', '83_1', '83_2', '84_1', '84_2', '85_1', '85_2', '86_1', '86_2']

print(len(files))

146


In [49]:
f = files[0]
path = '../../EEG original dataset/sub-AB{}/eeg/sub-AB{}_task-gonogo_run-{}_eeg.set'.format(f[0:-2], f[0:-2], f[-1])
test_raw = mne.io.read_raw_eeglab(path, eog=(), preload=True, uint16_codec=None, verbose=None)

In [58]:
data = np.concatenate([get_data(f) for f in files])
print(data)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 60.00 Hz
- Upper transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 67.50 Hz)
- Filter length: 3301 samples (6.602 sec)

Fitting ICA to data using 61 channels (please be patient, this may take a while)


<ipython-input-42-8b513a01f9e9>:3: DeprecationWarning: Version 0.23 introduced max_iter="auto", setting max_iter=1000 for `fastica` and max_iter=500 for `infomax` and `picard`. The current default of max_iter=200 will be changed to "auto" in version 0.24.
  ica = mne.preprocessing.ICA(n_components = 20, random_state = 42)


Selecting by number: 20 components
Fitting ICA took 10.4s.
    Searching for artifacts...
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    14, 14, 10
    Removing duplicate indices...
Ready.
Using threshold: 0.23 for CTPS ECG detection
Using channel EKG to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 sec)

Number of ECG events detected : 545 (average pulse 62 / min.)
Not setting metadata
Not setting metadata
545 matching events found
No baseline correction app

<ipython-input-42-8b513a01f9e9>:3: DeprecationWarning: Version 0.23 introduced max_iter="auto", setting max_iter=1000 for `fastica` and max_iter=500 for `infomax` and `picard`. The current default of max_iter=200 will be changed to "auto" in version 0.24.
  ica = mne.preprocessing.ICA(n_components = 20, random_state = 42)


Selecting by number: 20 components
Fitting ICA took 7.5s.
    Searching for artifacts...
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    7, 7, 4
    Removing duplicate indices...
Ready.
Using threshold: 0.23 for CTPS ECG detection
Using channel EKG to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 sec)

Number of ECG events detected : 535 (average pulse 60 / min.)
Not setting metadata
Not setting metadata
535 matching events found
No baseline correction applied

...

Selecting by number: 20 components
Fitting ICA took 6.8s.
    Searching for artifacts...
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    5, 5, 13
    Removing duplicate indices...
Ready.
Using threshold: 0.23 for CTPS ECG detection
Using channel EKG to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 sec)

Number of ECG events detected : 484 (average pulse 57 / min.)
Not setting metadata
Not setting metadata
484 matching events found
No baseline correction applie

<ipython-input-42-8b513a01f9e9>:3: DeprecationWarning: Version 0.23 introduced max_iter="auto", setting max_iter=1000 for `fastica` and max_iter=500 for `infomax` and `picard`. The current default of max_iter=200 will be changed to "auto" in version 0.24.
  ica = mne.preprocessing.ICA(n_components = 20, random_state = 42)


Selecting by number: 20 components
Fitting ICA took 8.7s.
    Searching for artifacts...
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    16, 16, 3
    Removing duplicate indices...
Ready.
Using threshold: 0.23 for CTPS ECG detection
Using channel EKG to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 sec)

Number of ECG events detected : 518 (average pulse 59 / min.)
Not setting metadata
Not setting metadata
518 matching events found
No baseline correction appli

<ipython-input-42-8b513a01f9e9>:3: DeprecationWarning: Version 0.23 introduced max_iter="auto", setting max_iter=1000 for `fastica` and max_iter=500 for `infomax` and `picard`. The current default of max_iter=200 will be changed to "auto" in version 0.24.
  ica = mne.preprocessing.ICA(n_components = 20, random_state = 42)


Selecting by number: 20 components
Fitting ICA took 11.8s.
    Searching for artifacts...
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    0, 0, 18
    Removing duplicate indices...
Ready.
Using threshold: 0.23 for CTPS ECG detection
Using channel EKG to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 sec)

Number of ECG events detected : 500 (average pulse 59 / min.)
Not setting metadata
Not setting metadata
500 matching events found
No baseline correction appli

In [60]:
print(data.shape)

(22565, 66, 351)


In [63]:
np.savez_compressed('data.npz', data)

In [ ]:
np.save('data_npy', data)

In [ ]:
#loaded_d = load('data.npz')